# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [41]:
city_data = pd.read_csv("../output_data/Weather.csv")
city_data = city_data.drop(columns=["Unnamed: 0"])
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tautira,-17.73,-149.15,78.80,88,40,5.82,PF,1592510906
1,Guiyang,26.58,106.72,68.00,94,90,2.24,CN,1592510907
2,Hong Kong,22.29,114.16,84.00,74,52,4.00,HK,1592510524
3,Mopti,14.67,-4.00,99.23,28,42,0.63,ML,1592510907
4,Dzhebariki-Khaya,62.22,135.80,60.48,72,97,3.76,RU,1592510907
...,...,...,...,...,...,...,...,...,...
582,Manaure,11.78,-72.44,96.80,44,20,17.22,CO,1592510862
583,Okato,-39.20,173.88,46.99,84,8,7.00,NZ,1592510998
584,Korsakov,46.63,142.78,46.40,87,75,15.66,RU,1592510998
585,Katobu,-4.94,122.53,75.06,93,76,5.17,ID,1592510744


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [42]:
# configure gmaps
gmaps.configure(api_key = g_key)
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

In [43]:
# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=np.max(humidity),
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [142]:
narrow_city_data = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & 
                                 (city_data["Wind Speed"] < 10)  & (city_data["Cloudiness"] == 0)].dropna()
narrow_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Oruzgan,32.83,66.00,70.95,20,0,5.46,AF,1592510911
78,Neckargemünd,49.39,8.79,73.40,46,0,3.36,DE,1592510918
102,Benghazi,32.12,20.07,75.52,53,0,5.32,LY,1592510921
164,Bangkalan,-7.05,112.74,78.91,84,0,5.61,ID,1592510930
249,Kāzerūn,29.62,51.65,79.50,19,0,0.25,IR,1592510943
431,Popondetta,-8.75,148.25,70.29,90,0,2.75,PG,1592510974
438,Luanda,-8.84,13.23,73.40,88,0,4.70,AO,1592510706
469,Mossamedes,-15.20,12.15,71.35,71,0,2.93,AO,1592510750
493,Goderich,43.75,-81.72,73.00,71,0,8.46,CA,1592510958
507,Pirgos,35.12,25.72,75.99,44,0,5.01,GR,1592510986


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [143]:
hotel_df = narrow_city_data.loc[:,["Country","City", "Lat", "Lng"]]
hotel_df["Hotel Name"]= ""
hotel_df

,Country,City,Lat,Lng,Hotel Name
33,AF,Oruzgan,32.83,66.00,
78,DE,Neckargemünd,49.39,8.79,
102,LY,Benghazi,32.12,20.07,
164,ID,Bangkalan,-7.05,112.74,
249,IR,Kāzerūn,29.62,51.65,
431,PG,Popondetta,-8.75,148.25,
438,AO,Luanda,-8.84,13.23,
469,AO,Mossamedes,-15.20,12.15,
493,CA,Goderich,43.75,-81.72,
507,GR,Pirgos,35.12,25.72,


In [144]:
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city= row["City"]
    params["location"] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel in {city} is: {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Couldnt find hotel. Skipping...")
        hotel_df.loc[index, "Hotel Name"] = ""

    print("-----------------------------")
print("Data Retrieval Complete")      
print("-----------------------------")

Retrieving Results for Index 33: Oruzgan.
Couldnt find hotel. Skipping...
-----------------------------
Retrieving Results for Index 78: Neckargemünd.
Closest hotel in Neckargemünd is: Hotel Neckarlux.
-----------------------------
Retrieving Results for Index 102: Benghazi.
Closest hotel in Benghazi is: Tibesti Hotel.
-----------------------------
Retrieving Results for Index 164: Bangkalan.
Closest hotel in Bangkalan is: VELLDAN Guest House.
-----------------------------
Retrieving Results for Index 249: Kāzerūn.
Closest hotel in Kāzerūn is: Bishapour Hotel.
-----------------------------
Retrieving Results for Index 431: Popondetta.
Closest hotel in Popondetta is: Comfort Inn.
-----------------------------
Retrieving Results for Index 438: Luanda.
Closest hotel in Luanda is: EPIC SANA Luanda.
-----------------------------
Retrieving Results for Index 469: Mossamedes.
Closest hotel in Mossamedes is: Hotel ibis Styles Iu Namibe.
-----------------------------
Retrieving Results for Inde

In [150]:
hotel_df = hotel_df[hotel_df["Hotel Name"]!='']
hotel_df

,Country,City,Lat,Lng,Hotel Name
78,DE,Neckargemünd,49.39,8.79,Hotel Neckarlux
102,LY,Benghazi,32.12,20.07,Tibesti Hotel
164,ID,Bangkalan,-7.05,112.74,VELLDAN Guest House
249,IR,Kāzerūn,29.62,51.65,Bishapour Hotel
431,PG,Popondetta,-8.75,148.25,Comfort Inn
438,AO,Luanda,-8.84,13.23,EPIC SANA Luanda
469,AO,Mossamedes,-15.20,12.15,Hotel ibis Styles Iu Namibe
493,CA,Goderich,43.75,-81.72,Samuels Hotel
507,GR,Pirgos,35.12,25.72,Mistral Mare Hotel
586,RU,Pallasovka,50.05,46.88,Otel'-El'


In [151]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [152]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))